# 패키지로딩

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score
#!pip install xlrd

# 하이퍼 파라미터 설정

In [2]:
INPUT_DIMS = 13
EPOCH=100
BATCH=32

# 데이터 로딩 및 확인

In [4]:
!pip install xlrd

DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
data = pd.read_excel('heart.xls')
print(data.shape)

(303, 14)


In [8]:
print(data.info())
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB
None


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


# 데이터 스케일링

In [10]:
from sklearn.preprocessing import StandardScaler

x=data.drop('target',axis=1)
y=data['target']
scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)

# 학습/검증/평가 데이터 분할

In [11]:
x_train, x_test, y_train, y_test = train_test_split(scaled_x,y,train_size=0.7,random_state=10)
x_train.shape, x_test.shape, y_train.shape,y_test.shape

((212, 13), (91, 13), (212,), (91,))

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=0)
x_train.shape, x_val.shape, y_train.shape,y_val.shape

((135, 13), (34, 13), (135,), (34,))

# 모델생성

In [17]:
from keras.layers import Dropout
from keras import regularizers
model = Sequential()
model.add(Dense(units=1000,activation='tanh',input_dim=INPUT_DIMS, kernel_regularizer=regularizers.L2(0.02)))
model.add(Dense(units=1000,activation='tanh',kernel_regularizer=regularizers.L2(0.1)))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1,activation='sigmoid'))
print(model.summary())

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 1000)                │          14,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1000)                │       1,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1000)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │           1,001 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,016,001 (3.88 MB)

 Trainable params: 1,016,001 (3.88 MB)

 Non-trainable params: 0 (0.00 B)

None


# 모델 컴파일 및 학습

## Tensorboard 사용준비

In [18]:
from tensorflow.keras.callbacks import TensorBoard
import datetime

log_dir = 'c:\Logs\\'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
# 히스토그램 1 에폭마다 활성화 함수의 출력 히스토그램 기록
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

## EarlyStopping 설정
EarlyStopping 매개변수
monitor : 모니터링 할 지표를 선택 'val_loss','val_acc' 등의 값을 지정
patience : 모니터링 지표의 값이 개선이 없다고 판단하기 전에 대기할 에폭수
mode : 모니터링 할 지표를 최소화할지 최대화 할지 결정
min : 

In [19]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',mode='min',patience=3)

In [20]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=BATCH, epochs=EPOCH, validation_data = (x_val,y_val),verbose=1, callbacks=[tensorboard,early_stop])

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 194ms/step - accuracy: 0.6239 - loss: 87.7630 - val_accuracy: 0.8824 - val_loss: 55.8816
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.8337 - loss: 51.5171 - val_accuracy: 0.7941 - val_loss: 37.5140
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.8092 - loss: 34.8010 - val_accuracy: 0.9118 - val_loss: 25.6356
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.8128 - loss: 23.9227 - val_accuracy: 0.9118 - val_loss: 17.5007
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.8693 - loss: 16.2750 - val_accuracy: 0.8824 - val_loss: 11.7572
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.8428 - loss: 10.9234 - val_accuracy: 0.9118 - val_loss: 7.6246
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.8217 - loss: 7.1042 - val_accuracy: 0.7647 - val_loss: 5.1168
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.8165 - loss: 4.6123 - val_accuracy: 0.8235

# 예측 및 모델 평가

In [27]:
y_pred_prob = model.predict(x_test)
# print(y_pred_prob[:5])
y_pred = (y_pred_prob > 0.5)
print(y_pred[:5])
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

score = model.evaluate(x_test,y_test)
print(f'Loss:{score[0]:.3f}')
print(f'Accuracy:{score[1]:.3f}')
print(f'Precision:{precision_score(y_test,y_pred):.3f}')
print(f'AUC:{roc_auc_score(y_test,y_pred_prob):.3f}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[[False]
 [False]
 [ True]
 [ True]
 [False]]
Confusion Matrix
[[38 12]
 [ 7 34]]
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7706 - loss: 0.7245 
Loss:0.726
Accuracy:0.791
Precision:0.739
AUC:0.849
